In [24]:
import wandb

In [25]:
wandb.login()

True

In [19]:
!python -W ignore ../src/train.py --tags "3#drop0.2" --base "efficientnet_b0" --image_size 128 --lr 1e-3 --epochs 200 \
--batch_size 36 --wandb 0

cuda
Using device: cuda
True
Tesla V100-PCIE-32GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
==> Preparing data
==> Preparing model
Using cache found in /headless/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /headless/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Adding attention
==> Initialising sampler
=====> Loading indices
=====> Loading samples
=====> Checking distribution
=====> Assigning weights
100%|███████████████████████████████████████| 2565/2565 [08:54<00:00,  4.80it/s]
==> Training model
  0%|                                                    | 0/36 [00:08<?, ?it/s]
Traceback (most recent call last):
  File "../src/train.py", line 382, in <module>
    main()
  File "../src/train.py", line 376, in main
    train_loop(args)
  File "../src/train.py", line 338, in train_loop
    test_len = test(model, DEVICE, test_loader, loss, epoch, num_classes=NUM_CLASSES, wandb_log=WANDB)
  File "../src/train.py", line 211, in test
  

In [18]:
import torch
import sys
sys.path.append('../src/')
from attention_augmented_conv import AugmentedConv

model = torch.nn.Sequential(
                      AugmentedConv(in_channels=1280, out_channels=6, kernel_size=1, dk=40, dv=4, Nh=1, relative=False, stride=2),
                     #nn.Conv2d(588, 6, kernel_size=1, padding = 1, stride=1, bias=False),
                     torch.nn.BatchNorm2d(6), 
                     torch.nn.Dropout(p=0.25),
                     torch.nn.AdaptiveAvgPool2d(1)
                      ).to("cuda")
temp_input = torch.randn((16, 1280, 32, 32)).to('cuda')
augmented_conv = AugmentedConv(in_channels=1280, out_channels=6, kernel_size=1, dk=40, dv=4, Nh=1, relative=False, stride=2).to('cuda')
norm = torch.nn.BatchNorm2d(6).to("cuda")

conv_out = model(temp_input).squeeze()
print(conv_out.shape)

torch.Size([16, 6])


In [4]:
!python -W ignore ../src/train.py --tags "1#drop0.2" --base "mixnet_s" --image_size 224 --wandb 1 \
          --batch_size 24 --inp_size 1536

cuda
Using device: cuda
True
Tesla V100-PCIE-32GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
wandb!
wandb: Tracking run with wandb version 0.8.18
wandb: Run data is saved locally in ../../../shared/kern_segmentation/logs/wandb/run-20191207_222740-rqsiwlo7
wandb: Syncing run 1#drop0.2_mixnet_s_ft_0.001_6
wandb: ⭐️ View project at https://app.wandb.ai/nuvard/kern
wandb: 🚀 View run at https://app.wandb.ai/nuvard/kern/runs/rqsiwlo7
wandb: Run `wandb off` to turn off syncing.

==> Preparing data
==> Preparing model
Using cache found in /headless/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /headless/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mixnet_s-a907afbc.pth" to /headless/.cache/torch/checkpoints/mixnet_s-a907afbc.pth
^C
wandb: Ctrl-c pressed.
wandb: Program failed with code 255. Press ctrl-c to abort syncing.
Traceback (most recent ca

In [ ]:
!python -W ignore ../src/train.py --tags "1#drop0.2#" --base "mnasnet_a1" --image_size 224 --wandb 1